In [8]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table
from dash.dependencies import Input, Output, State
import base64
import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from mongodbCRUD import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "password"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object and accept projection json input
df = pd.DataFrame.from_records(shelter.read({}))





#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')
# Added in Grazioso Salvare’s logo
image_filename = 'Grazioso _Salvare _Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())


app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
# places the picture as part of the title while making the picture a hyperlink to a web address   
    dcc.Link(
        href="https://www.snhu.edu",
        refresh=True,
        children=[html.Center(html.A(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))))]
    ),
    #the rest of the title provided here 
    html.Center(html.B(html.H1('Animal Shelter Web Dashboard', style={'height': '40px', 'padding':'10px',
                                           'color':'red'}))),
    html.Hr(),
    
    #this set the button up above the data chart.
    html.Div(className='row',
             style={'display' : 'flex'},
                  children=[
                        html.Button('Water Rescue',id='submit-button-one',n_clicks=0),
                        html.Button('Mountain Rescue',id='submit-button-two',n_clicks=0),
                        html.Button('Disaster Rescue',id='submit-button-three',n_clicks=0),
                        html.Button('Reset',id='submit-button-four',n_clicks=0),
                        
                  ]),
    # Code for data table
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
    # this Sets up the features for your interactive data table to make it user-friendly for your client
        editable = False,
        
        sort_action = 'native',
        sort_mode = 'multi',
        column_selectable = False,
        row_selectable = 'single',
        row_deletable = False,
        selected_columns = [],
        selected_rows = [],
        page_action = 'native',
        page_current = 0,
        page_size = 10,
    ),
    
    #code that sets up the locations for graph and map 
    html.Br(),
     html.Hr(),
     html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ]),
    #includes the author credit at the end of page
    html.Div(html.H4('Created by Jeremia Faust', style={'height': '20px', 'padding':'5px',
                                           'text-align':'right','color':'green'}))
    
])

#############################################
# Interaction Between Components / Controller
#############################################
#callback to proved functionality to buttons by making connections between method and the clickable buttons
@app.callback(Output('datatable-id',"data"), 
              [Input('submit-button-one', 'n_clicks'),
               Input('submit-button-two', 'n_clicks'),
               Input('submit-button-three', 'n_clicks'),
               Input('submit-button-four', 'n_clicks'),
               
              ])
#this the method that tells what the buttons do 
def on_click(bt1,bt2,bt3,bt4):
    # start case
    if (int(bt1) == 0 and int(bt2) == 0 and int(bt3) and int(bt4)):
        df = pd.DataFrame.from_records(shelter.read({}))
    # use higher number of button clicks to determine filter type
    #button for water rescue
    if (int(bt1) > int(bt2) and int(bt1) > int(bt3) and int(bt1) > int(bt4) ):
        df = pd.DataFrame.from_records(shelter.read({'$and':[{'$or': 
                            [{'breed':'Labrador Retriever Mix'},
                             {'breed' : "Chesapeake Bay Retriver"},
                            {'breed' : 'Newfoundland'}]},
                            {'sex_upon_outcome' : 'Intact Female'},
                             {'age_upon_outcome_in_weeks' : {'$gte':26,'$lte':156}}]}))
        bt2=0
    #Button for mountain rescue
    if (int(bt2) > int(bt1) and int(bt2) > int(bt3) and int(bt2) > int(bt4) ):
        df = pd.DataFrame.from_records(shelter.read({'animal_type':'Dog','breed':{'$in':[
                                'Geman Shepherd',
                                "Alaskan Malamute",
                                'Old English Sheepdog',
                                "Siberian Husky",
                                "Rottweiler"]},
                                'sex_upon_outcome' : 'Intact Male',
                                'age_upon_outcome_in_weeks':{'$gte':26},
                                'age_upon_outcome_in_weeks' : {'$lte':156}})) 
        b1=0
    #button for disaster rescue
    if (int(bt3) > int(bt1) and int(bt3) > int(bt2) and int(bt3) > int(bt4) ):
        df = pd.DataFrame.from_records(shelter.read({'animal_type':'Dog','breed':{'$in':[
                                'Geman Shepherd',
                                'Doberman Pinscher',
                                'Golden Retriever',
                                "Bloodhound",
                                "Rottweiler"]},
                                'sex_upon_outcome' : 'Intact Male',
                                'age_upon_outcome_in_weeks':{'$gte':20},
                                'age_upon_outcome_in_weeks' : {'$lte':300}})) 
        b3=0
    #button for reset    
    if (int(bt4) > int(bt1) and int(bt4) > int(bt2) and int(bt4) > int(bt3) ):
            df = pd.DataFrame.from_records(shelter.read({}))
            b4=0
        
    return df.to_dict('records')


#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

#this callback will create a bar graph
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])

def update_graphs(viewData):
    
    df = pd.DataFrame.from_dict(viewData)
    return [
        dcc.Graph( 
        id=column,
           figure ={"data":[{"x":df['breed'],"type":"bar","marker":{"color":"purple"},}],
            "layout":{
                "title": "Total Breeds",
                "height":500,
                "margin":{"t":40,"l":40,"r":40},
            },
        },  
       )
        for column in["breed"]
        if column in df
    ]
#callback to create a geolocation chart
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])


def update_map(viewData):
    
    dff = pd.DataFrame.from_dict(viewData)
    
    
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[dff.loc[0,'location_lat'],
                                                                     dff.loc[0,'location_long']], 
                                                                       zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[dff.loc[0,'location_lat'],dff.loc[0,'location_long']], children=[
                dl.Tooltip(dff['breed']),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.loc[0,'name'])
                ])
            ])
        ])
    ]    

app